In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch.nn.functional as F
import torch
from torch import nn
from torchsummary import summary
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

from text_recognizer.networks.transformer.vit import ViT
from text_recognizer.networks.transformer.transformer import Transformer
from text_recognizer.networks.transformer.layers import Decoder

In [2]:
torch.cuda.is_available()

True

In [3]:
loss = nn.CrossEntropyLoss()

In [51]:
o = torch.randn((4, 5, 4))

In [52]:
t = torch.randint(0, 5, (4, 4))

In [53]:
o.shape

torch.Size([4, 5, 4])

In [54]:
t.shape

torch.Size([4, 4])

In [55]:
t

tensor([[0, 1, 3, 2],
        [1, 4, 4, 4],
        [1, 4, 2, 1],
        [2, 0, 4, 4]])

In [56]:
o

tensor([[[ 0.0647, -1.3831,  0.0266,  0.8528],
         [ 1.4976,  0.4153,  1.0353,  0.0154],
         [ 1.4562, -0.3568,  0.3599, -0.6222],
         [ 0.2773,  0.4563,  0.9282, -2.1445],
         [ 0.5191,  0.3683, -0.3469,  0.1355]],

        [[ 0.0424, -0.3215,  0.5662, -0.4217],
         [ 2.0793,  1.2817,  0.1559, -0.6900],
         [-1.1751, -0.3359,  1.7875, -0.3671],
         [-0.4553, -0.3952, -0.8633,  0.1538],
         [-1.3862,  0.4255, -2.2948,  0.0312]],

        [[-1.4257,  2.2662,  0.2670, -0.4330],
         [-0.3244, -0.8669, -0.2571,  0.8028],
         [ 0.9109, -0.2289, -1.2095, -0.9761],
         [-0.0156,  1.2403, -1.1967,  0.6841],
         [-0.8185,  0.2967, -2.1639, -0.7903]],

        [[-1.0425,  0.1426,  0.1383,  0.9784],
         [-1.2853,  1.4123, -0.2272, -0.3335],
         [ 1.5751, -0.7663,  0.9610,  0.5686],
         [ 0.9697, -1.5515, -0.8658, -0.5882],
         [-1.2467,  0.0539,  0.1208, -1.0297]]])

In [57]:
loss(o, t)

tensor(1.8355)

In [63]:
t[:, 2] == 2 | t[:, 2] == 1

TypeError: unsupported operand type(s) for |: 'int' and 'Tensor'

In [60]:
torch.argmax(o, dim=-1)[:, -1:].shape

torch.Size([4, 1])

In [43]:
class LabelSmoothingLossCanonical(nn.Module):
    def __init__(self, smoothing=0.0, dim=-1):
        super(LabelSmoothingLossCanonical, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            print(true_dist.shape)
            true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
            print(true_dist.shape)
            print(true_dist)
            true_dist.masked_fill_((target == 4).unsqueeze(1), 0)
            print(true_dist)
            true_dist += self.smoothing / pred.size(self.dim)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [44]:
l = LabelSmoothingLossCanonical(0.1)

In [45]:
l(o, t)

torch.Size([1, 5, 4])
torch.Size([1, 5, 4])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.9000, 0.9000, 0.0000, 0.9000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.9000, 0.0000]]])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.9000, 0.9000, 0.0000, 0.9000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])


tensor(0.9438)

In [6]:
from text_recognizer.networks.encoders.efficientnet.efficientnet import EfficientNet

In [7]:
en = EfficientNet("b0")

In [9]:
def generate_square_subsequent_mask(size: int) -> torch.Tensor:
    """Generate a triangular (size, size) mask."""
    mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask == 1, float(0.0))
    return mask

In [10]:
generate_square_subsequent_mask(4)

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [12]:
from torch import Tensor

In [52]:
tgt = torch.randint(0, 4, (1, 4))
tgt_mask = torch.ones_like(tgt).bool()

In [53]:
tgt_mask

tensor([[True, True, True, True]])

In [13]:
def target_padding_mask(trg: Tensor, pad_index: int) -> Tensor:
    """Returns causal target mask."""
    trg_pad_mask = (trg != pad_index)[:, None, None]
    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=trg.device)).bool()
    trg_mask = trg_pad_mask & trg_sub_mask
    return trg_mask

In [54]:
t = torch.randint(0, 6, (0, 4))

In [55]:
t = torch.Tensor([[0, 0, 0, 3, 3, 3]])

In [58]:
tt = t != 3

In [59]:
tt

tensor([[ True,  True,  True, False, False, False]])

In [43]:
t = torch.cat((t, t))

In [44]:
t.shape

torch.Size([2, 6])

In [45]:
target_padding_mask(t, 3)

tensor([[[[ True, False, False, False, False, False],
          [ True,  True, False, False, False, False],
          [ True,  True,  True, False, False, False],
          [ True,  True,  True, False, False, False],
          [ True,  True,  True, False, False, False],
          [ True,  True,  True, False, False, False]]],


        [[[ True, False, False, False, False, False],
          [ True,  True, False, False, False, False],
          [ True,  True,  True, False, False, False],
          [ True,  True,  True, False, False, False],
          [ True,  True,  True, False, False, False],
          [ True,  True,  True, False, False, False]]]])

In [ ]:
target_padding_mask()

In [ ]:
summary(en, (1, 224, 224));

In [ ]:
torch.cuda.is_available()

In [ ]:
decoder = Decoder(dim=128, depth=2, num_heads=8, ff_kwargs={}, attn_kwargs={}, cross_attend=True)

In [ ]:
decoder.cuda()

In [ ]:
transformer_decoder = Transformer(num_tokens=1003, max_seq_len=451, attn_layers=decoder, emb_dim=128, emb_dropout=0.1)

In [ ]:
transformer_decoder.cuda()

In [ ]:
efficient_transformer = Nystromer(
    dim = 64,
    depth = 4,
    num_heads = 8,
    num_landmarks = 64
)

In [ ]:
v = ViT(
    dim = 64,
    image_size = (576, 640),
    patch_size = (32, 32),
    transformer = efficient_transformer
).cuda()

In [6]:
t = torch.randn(4, 1, 576, 640).cuda()

In [7]:
en.cuda()

EfficientNet(
  (_conv_stem): Sequential(
    (0): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    (1): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (2): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (3): Mish(inplace=True)
  )
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
        (2): Mish(inplace=True)
      )
      (_squeeze_excite): Sequential(
        (0): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
        (1): Mish(inplace=True)
        (2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (_pointwise): Sequential(
        (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      )
    )

In [8]:
en(t).shape

torch.Size([4, 1280, 18, 20])

In [ ]:
o = v(t)

In [ ]:
caption = torch.randint(0, 90, (16, 690)).cuda()

In [ ]:
o.shape

In [ ]:
caption.shape

In [ ]:
o = torch.randn(16, 20 * 18, 128).cuda()

In [ ]:
caption = torch.randint(0, 1000, (16, 200)).cuda()

In [ ]:
transformer_decoder(caption, context = o).shape # (1, 1024, 20000)

In [ ]:
efficient_transformer = efficient_transformer(num_landmarks=256)

In [ ]:
efficient_transformer()

In [ ]:
from omegaconf import OmegaConf

In [ ]:
path = "../training/configs/vqvae.yaml"

In [ ]:
conf = OmegaConf.load(path)

In [ ]:
print(OmegaConf.to_yaml(conf))

In [ ]:
from text_recognizer.networks import VQVAE

In [ ]:
vae = VQVAE(**conf.network.args)

In [ ]:
vae

In [ ]:
datum = torch.randn([2, 1, 576, 640])

In [ ]:
vae.encoder(datum)[0].shape

In [ ]:
vae(datum)[0].shape

In [ ]:
datum = torch.randn([2, 1, 576, 640])

In [ ]:
trg = torch.randint(0, 1000, [2, 682])

In [ ]:
trg.shape

In [ ]:
datum = torch.randn([2, 1, 224, 224])

In [ ]:
en(t).shape

In [ ]:
path = "../training/configs/cnn_transformer.yaml"

In [ ]:
conf = OmegaConf.load(path)

In [ ]:
print(OmegaConf.to_yaml(conf))

In [ ]:
from text_recognizer.networks.cnn_transformer import CNNTransformer

In [ ]:
t = CNNTransformer(input_shape=(1, 576, 640), output_shape=(682, 1), **conf.network.args)

In [ ]:
t.encode(datum).shape

In [ ]:
trg.shape

In [ ]:
t(datum, trg).shape

In [ ]:
b, n = 16, 128
device = "cpu"

In [ ]:
x = lambda: torch.ones((b, n), device=device).bool()

In [ ]:
x().shape

In [ ]:
torch.ones((b, n), device=device).bool().shape

In [ ]:
x = torch.randn(1, 1, 576, 640)

In [ ]:
576 // 32

In [ ]:
640 // 32

In [ ]:
18 * 20

In [ ]:
x = torch.randn(1, 1, 144, 160)

In [ ]:
from einops import rearrange

In [ ]:
patch_size=16
p = rearrange(x, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size)

In [ ]:
p.shape